### 02. 문자 단위 RNN - 더 많은 데이터

#### 2. 문자 단위 RNN

In [35]:
# 필요한 도구 임포트
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

##### 1. 훈련 데이터 전처리

In [26]:
sentence = ("if you want to build a ship, don't drum up people together to "
            "collect wood and don't assign them tasks and work, but rather "
            "teach them to long for the endless immensity of the sea.")

In [27]:
# 문자 집합 생성, 각 문자에 고유한 정수 부여
char_set = list(set(sentence)) # 중복 제거한 문자 집합 생성
char_dic = {c: i for i, c in enumerate(char_set)}
print(char_set)
print(char_dic)

["'", 'e', 'u', ',', 'n', 'i', 'd', 'a', 'o', 'f', 'm', 'c', 'k', 'l', ' ', 'r', 'y', 't', '.', 'g', 'w', 'p', 'b', 'h', 's']
{"'": 0, 'e': 1, 'u': 2, ',': 3, 'n': 4, 'i': 5, 'd': 6, 'a': 7, 'o': 8, 'f': 9, 'm': 10, 'c': 11, 'k': 12, 'l': 13, ' ': 14, 'r': 15, 'y': 16, 't': 17, '.': 18, 'g': 19, 'w': 20, 'p': 21, 'b': 22, 'h': 23, 's': 24}


In [28]:
# 총 25개의 문자 존재
# 문자 집합의 크기 확인
dic_size = len(char_dic)
print('문자 집합의 크기 : {}'.format(dic_size))

문자 집합의 크기 : 25


- 문자 집합의 크기는 25이며, 입력을 원-핫 벡터로 사용할 것이므로 매 시점마다 들어갈 입력의 크기이기도 한다.
- 하이퍼 파라미터 설정
    - hidden_size를 입력의 크기와 도일하게 줬는데, 다른 값을 줘도 무방
    - sequence_length라는 변수 선언 : 앞서 만든 샘플을 10개 단위로 끊어서 샘플을 만들 예정

In [29]:
# 하이퍼파라미터 설정
hidden_size = dic_size
sequence_length = 10
learning_rate = 0.1

In [30]:
sentence[100]

'k'

In [31]:
# sequence_length 값인 10의 단위로 샘플들을 잘라서 데이터를 만드는 모습
x_data = []
y_data = []

for i in range(0, len(sentence)-sequence_length) : 
    x_str = sentence[i : i+sequence_length]
    y_str = sentence[i+1 : i+sequence_length+1]
    print(i, x_str,'->', y_str)
    
    x_data.append([char_dic[c] for c in x_str])
    y_data.append([char_dic[c] for c in y_str])

0 if you wan -> f you want
1 f you want ->  you want 
2  you want  -> you want t
3 you want t -> ou want to
4 ou want to -> u want to 
5 u want to  ->  want to b
6  want to b -> want to bu
7 want to bu -> ant to bui
8 ant to bui -> nt to buil
9 nt to buil -> t to build
10 t to build ->  to build 
11  to build  -> to build a
12 to build a -> o build a 
13 o build a  ->  build a s
14  build a s -> build a sh
15 build a sh -> uild a shi
16 uild a shi -> ild a ship
17 ild a ship -> ld a ship,
18 ld a ship, -> d a ship, 
19 d a ship,  ->  a ship, d
20  a ship, d -> a ship, do
21 a ship, do ->  ship, don
22  ship, don -> ship, don'
23 ship, don' -> hip, don't
24 hip, don't -> ip, don't 
25 ip, don't  -> p, don't d
26 p, don't d -> , don't dr
27 , don't dr ->  don't dru
28  don't dru -> don't drum
29 don't drum -> on't drum 
30 on't drum  -> n't drum u
31 n't drum u -> 't drum up
32 't drum up -> t drum up 
33 t drum up  ->  drum up p
34  drum up p -> drum up pe
35 drum up pe -> rum up peo
36

In [33]:
# 총 170개의 샘플 생성
# 첫번째 샘플의 입력 데이터와 레이블 데이터 출력
print(x_data[0])
print(y_data[0])

[5, 9, 14, 16, 8, 2, 14, 20, 7, 4]
[9, 14, 16, 8, 2, 14, 20, 7, 4, 17]


- 한 칸씩 shift된 시퀀스가 정상적으로 출력
- 이제 입력 시퀀스에 대해서 원-핫 인코딩을 수행하고 입력 데이터와 레이블 데이터를 텐서로 변환

In [36]:
x_one_hot = [np.eye(dic_size)[x] for x in x_data]
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

C:\Users\ATIV\AppData\Local\Temp\ipykernel_18968\2907341166.py:2: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ..\torch\csrc\utils\tensor_new.cpp:210.)
  X = torch.FloatTensor(x_one_hot)


In [37]:
# 크기 확인
print('훈련 데이터의 크기 : {}'.format(X.shape))
print('레이블의 크기 : {}'.format(Y.shape))

훈련 데이터의 크기 : torch.Size([170, 10, 25])
레이블의 크기 : torch.Size([170, 10])


In [39]:
# 첫번째 샘플 출력
print(X[0])
print(Y[0])

tensor([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,

----------------------------------------------------------------------------------------------------------------

#### 2. 모델 구현

In [41]:
class Net(torch.nn.Module) : 
    def __init__(self, input_dim, hidden_dim, layers) : 
        super(Net, self).__init__()
        self.rnn = torch.nn.RNN(input_dim, hidden_dim, num_layers = layers, batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, hidden_dim, bias=True)
    def forward(self, x) : 
        x, _status = self.rnn(x)
        x = self.fc(x)
        return x

In [42]:
net = Net(dic_size, hidden_size, 2)

In [43]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), learning_rate)

In [55]:
# 모델에 입력을 넣어서 출력의 크기 확인
outputs = net(X)
print(outputs.shape)
print(dic_size)

torch.Size([170, 10, 25])
25


In [56]:
print(outputs)

tensor([[[ 0.3520,  0.1670,  0.0888,  ...,  0.0620,  0.0141,  0.1175],
         [ 0.2913,  0.3082, -0.0253,  ...,  0.0509, -0.0960,  0.1894],
         [ 0.3048,  0.2949, -0.0277,  ..., -0.0042, -0.1717,  0.1052],
         ...,
         [ 0.3394,  0.2456,  0.0175,  ...,  0.0202, -0.0716,  0.1987],
         [ 0.2765,  0.2246,  0.0389,  ..., -0.0394, -0.0490,  0.1047],
         [ 0.2451,  0.2668,  0.0698,  ..., -0.0321, -0.0518,  0.1361]],

        [[ 0.3821,  0.2212,  0.0324,  ...,  0.0315, -0.0412,  0.1962],
         [ 0.3136,  0.2925, -0.0167,  ..., -0.0125, -0.1989,  0.1736],
         [ 0.3174,  0.2750, -0.0371,  ...,  0.0495, -0.1279,  0.1084],
         ...,
         [ 0.2772,  0.2241,  0.0399,  ..., -0.0385, -0.0489,  0.1043],
         [ 0.2446,  0.2664,  0.0690,  ..., -0.0315, -0.0514,  0.1363],
         [ 0.3095,  0.2996,  0.0891,  ..., -0.0871, -0.0414,  0.1463]],

        [[ 0.3797,  0.2004,  0.0419,  ...,  0.0299, -0.0796,  0.1807],
         [ 0.3442,  0.2722, -0.0432,  ...,  0

In [53]:
print(outputs.view(-1, 25).shape)

torch.Size([1700, 25])


In [48]:
# 레이블의 크기 다시 확인
print(Y.shape)
print(Y.view(-1).shape)

torch.Size([170, 10])
torch.Size([1700])


In [60]:
for i in range(100) : 
    optimizer.zero_grad()
    outputs = net(X) # (170, 10, 25)크기를 가진 텐서를 매 에포크마다 모델의 입력으로 사용
    loss = criterion(outputs.view(-1, dic_size), Y.view(-1))
    loss.backward()
    optimizer.step()
    
    # result의 텐서 크기는 (170, 10)
    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results) : 
        # 처음에는 예측 결과를 전부 가져옴
        if j==0 : 
            predict_str += ''.join([char_set[t] for t in result])
        # 그 다음에는 마지막 글자만 반복 추가
        else : 
            predict_str += char_set[result[-1]]
            
    print(predict_str)

'e''''e'pe'e''e''e'e''''i'e'ke'e'eee''e''e'e''eee'e'eee'e'e''ee'eee'ee'''eeeee'e'e'ee''ek''e''e'e'''i'epee''''''e''e''''ne'e'e''''''e'e''eeeeee'''e'eeeeie'e'ei'''ee'n''e'ee''ee'ep
oo h  hhhhhhh hh  hhhhhhhhh hhh hhhhh  h  hh     hh h h   hh hh  h   hh  hhh hhhh hhhhhhhh hh h hh hhhhhhhhh  h hh  hhhh h hh h  hh h hh hh  hhh  hh hhh hh  hhh    hhh h  hh hhh h
                                                                                                                                                                                   
nb.. nl..         a  a             a    .a   a               a            aaa   aa   aa   aa       a  a a        a    aa         a       a         a    a    a   a a    aa    a    
tsssilssimllmlmlllllmmmmlmmlmlllolllllmlilmlmllmmmllmlllmlmlmmlllllllllmmlmmlmmmmlmlllmlmllllllsllmlllllllmmmlllllmlmmmimllmlllllmlmmlllmlmlllllmlllllmlmlmmllmlllmlllllmilllnlllmm
ttssittsotttttttttttttttttttttttttttttttitttttitttttttttttttttttttttstttttttttttttttttslttittttttttt